In [1]:
import pandas as pd

In [2]:
import math

In [3]:
MCI_flights_df = pd.read_csv('../Resources/MCI_21-23.csv').drop(['Unnamed: 0'],axis=1)

In [ ]:
MCI_flights_df=MCI_flights_df.replace({'Status': {'On Time': 0, 'Delayed': 1},'MONTH':{1:'January', 2: 'February',3:'March', 4: 'April',5:'May', 6: 'Jun',7:'July', 8: 'August',9:'September', 10: 'October',11:'November', 12: 'December'},'DAY_OF_WEEK':{1:'Monday', 2: 'Tuesday',3:'Wednesday', 4: 'Thursday',5:'Friday', 6: 'Saturday',7:'Sunday'}})
MCI_flights_df

In [ ]:
MCI_flights_df = MCI_flights_df.astype({'DAY_OF_MONTH': 'object','DEP_DELAY_GROUP':'object'})

In [ ]:
MCI_flights_df.columns

In [ ]:
MCI_flights_df['Status'].describe()

In [ ]:
MCI_flights_df.loc[MCI_flights_df['MKT_UNIQUE_CARRIER']=='WN']['Status'].value_counts()

MCI_flights_df.to_csv('../Resources/MCI_21-23.csv')

In [ ]:
MCI_origin_df=MCI_flights_df.loc[MCI_flights_df['ORIGIN'] == 'MCI']
MCI_dest_df=MCI_flights_df.loc[MCI_flights_df['DEST'] == 'MCI']
MCI_2021_df=MCI_flights_df.loc[MCI_flights_df['YEAR'] == 2021]
MCI_2022_df=MCI_flights_df.loc[MCI_flights_df['YEAR'] == 2022]
MCI_2023_df=MCI_flights_df.loc[MCI_flights_df['YEAR'] == 2023]
MCI_WN_df=MCI_flights_df.loc[MCI_flights_df['MKT_UNIQUE_CARRIER'] == 'WN']
MCI_notWN_df=MCI_flights_df.loc[MCI_flights_df['MKT_UNIQUE_CARRIER'] != 'WN']

In [ ]:
avg_dept_delay_dict = MCI_origin_df.groupby(['MONTH', 'DAY_OF_WEEK', 'MKT_UNIQUE_CARRIER', 'DEP_TIME_BLK','DEST'])['DEP_DELAY_NEW'].mean()
avg_arr_delay_dict = MCI_dest_df.groupby(['MONTH','DAY_OF_WEEK', 'MKT_UNIQUE_CARRIER', 'ORIGIN'])['ARR_DELAY_NEW'].mean()

In [ ]:
avg_arr_delay_dict

In [ ]:
avg_dept_delay_dict.describe()

In [ ]:
avg_dept_delay_dict

In [ ]:
avg_origin_dep_delays = []
for row in MCI_origin_df.iterrows():
    month = row[1]['MONTH']
    dow = row[1]['DAY_OF_WEEK']
    carrier = row[1]['MKT_UNIQUE_CARRIER']
    time = row[1]['DEP_TIME_BLK']
    dest = row[1]['DEST']
    try:
        avg_dep_delay = avg_dept_delay_dict[month][dow][carrier][time][dest]
        avg_origin_dep_delays.append(avg_dep_delay)
    except:
        avg_origin_dep_delays.append(12.72)

In [ ]:
avg_dest_dep_delays = []
for row in MCI_dest_df.iterrows():
    month = row[1]['MONTH']
    dow = row[1]['DAY_OF_WEEK']
    carrier = row[1]['MKT_UNIQUE_CARRIER']
    time = row[1]['DEP_TIME_BLK']
    origin = row[1]['ORIGIN']
    try:
        avg_dep_delay = avg_dept_delay_dict[month][dow][carrier][time][origin]
        avg_dest_dep_delays.append(avg_dep_delay)
    except:
        avg_dest_dep_delays.append(12.72)

In [ ]:
avg_origin_arr_delays = []
for row in MCI_origin_df.iterrows():
    month = row[1]['MONTH']
    dow = row[1]['DAY_OF_WEEK']
    carrier = row[1]['MKT_UNIQUE_CARRIER']
    dest = row[1]['DEST']
    try:
        avg_arr_delay = avg_arr_delay_dict[month][dow][carrier][dest]
        avg_origin_arr_delays.append(avg_arr_delay)
    except:
        avg_origin_arr_delays.append(12)

In [ ]:
avg_dest_arr_delays = []
for row in MCI_dest_df.iterrows():
    month = row[1]['MONTH']
    dow = row[1]['DAY_OF_WEEK']
    carrier = row[1]['MKT_UNIQUE_CARRIER']
    origin = row[1]['ORIGIN']
    try:
        avg_arr_delay = avg_arr_delay_dict[month][dow][carrier][origin]
        avg_dest_arr_delays.append(avg_arr_delay)
    except:
        avg_dest_arr_delays.append(12)

In [ ]:
MCI_origin_df['AVG_DEP_DELAY']=avg_origin_dep_delays
MCI_origin_df['AVG_ARR_DELAY']=avg_origin_arr_delays

MCI_dest_df['AVG_DEP_DELAY']=avg_dest_dep_delays
MCI_dest_df['AVG_ARR_DELAY']=avg_dest_arr_delays

In [ ]:
MCI_origin_df

In [ ]:
MCI_dest_df

In [ ]:
X_origin = MCI_origin_df[['YEAR','MONTH','DAY_OF_MONTH', 'DAY_OF_WEEK', 'MKT_UNIQUE_CARRIER', 'ORIGIN', 'DEST', 'DEP_TIME_BLK']]

In [ ]:
X_dest = MCI_dest_df[['YEAR','MONTH','DAY_OF_MONTH', 'DAY_OF_WEEK', 'MKT_UNIQUE_CARRIER', 'ORIGIN', 'DEST', 'DEP_TIME_BLK']]

In [ ]:
X = pd.concat([X_origin,X_dest])
X

In [ ]:
X['DELAY_FACTOR'] = X['AVG_ARR_DELAY']*X['AVG_DEP_DELAY']

In [ ]:
factors = []
for factor in X['DELAY_FACTOR']:
    factors.append(math.sqrt(factor))

In [ ]:
X['DELAY_FACTOR'] = factors

In [ ]:
X.dtypes

In [ ]:
X.describe()

In [ ]:
avg_arr_delay_blks = []
avg_dep_delay_blks = []
avg_delay_blks = []    
avg_taxi_blks = []
for row in X.iterrows():
    avg_arr_delay = row[1]['AVG_ARR_DELAY']
    avg_dep_delay = row[1]['AVG_DEP_DELAY']
    avg_delay = row[1]['DELAY_FACTOR']
    
    if avg_arr_delay <= 7.180000:
        avg_arr_delay_blk = '0-25%'
    elif avg_arr_delay <= 11.859649:
        avg_arr_delay_blk = '25-50%'
    elif avg_arr_delay <= 18.013158:
        avg_arr_delay_blk = '50-75%'
    else:
        avg_arr_delay_blk = '75-100%'
    avg_arr_delay_blks.append(avg_arr_delay_blk)
    
    if avg_dep_delay <= 2.000000:
        avg_dep_delay_blk = '0-25%'
    elif avg_dep_delay < 12.720000:
        avg_dep_delay_blk = '25-50%'
    elif avg_dep_delay == 12.720000:
        avg_dep_delay_blk = '50-75%'
    else:
        avg_dep_delay_blk = '75-100%'
    avg_dep_delay_blks.append(avg_dep_delay_blk)
    
    if avg_delay <= 2.300502:
        avg_delay_blk = '0-25%'
    elif avg_delay <= 7.321202:
        avg_delay_blk = '25-50%'
    elif avg_delay <= 14.838485:
        avg_delay_blk = '50-75%'
    else:
        avg_delay_blk = '75-100%'
    avg_delay_blks.append(avg_delay_blk)    

In [ ]:
X['AVG_ARR_DELAY_GROUPS']=avg_arr_delay_blks
X['AVG_DEP_DELAY_GROUPS']=avg_arr_delay_blks
X['DELAY_FACTOR_GROUPS']=avg_delay_blks

X

In [ ]:
X_origin = X.loc[X['ORIGIN']=='MCI']
X_dest = X.loc[X['DEST']=='MCI']

In [ ]:
MCI_origin_df.columns

In [ ]:
y_origin = MCI_origin_df['Status']
y_dest = MCI_dest_df['Status']

In [ ]:
y_dest

In [ ]:
y = pd.concat([y_origin,y_dest])

In [ ]:
y.dtypes

In [ ]:
y_dummies = pd.get_dummies(y)

In [ ]:
y_dummies.describe()

In [ ]:
X.columns

In [ ]:
X_dummies = pd.get_dummies(X[['MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'MKT_UNIQUE_CARRIER',
       'ORIGIN', 'DEST', 'DEP_TIME_BLK']], drop_first=True)

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from keras.layers import Dense, Dropout

In [ ]:
scaler = StandardScaler()
scaler.fit(X_dummies)
transformed_data = scaler.transform(X_dummies)
# Creating a DataFrame with with the scaled data
transformed_data_df = pd.DataFrame(transformed_data)

# Display sample data
transformed_data_df

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np
pca = PCA().fit(transformed_data_df)
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')

In [ ]:
from sklearn.decomposition import PCA
# Create a PCA model
nc = 15
pca = PCA(n_components=nc)
pca_data = pca.fit_transform(transformed_data_df)
pca_dict = {}
for i in range(0,nc):
    pca_dict[f'pca{i+1}']=pca_data[:,i]
# Display sample data
pca_df = pd.DataFrame(pca_dict)
pca_df

In [ ]:
y

In [ ]:
X_dummies.columns

In [ ]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X_dummies, y,test_size=0.30, random_state=78)

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=64, activation="relu", input_dim=len(X_dummies.columns)))
# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=128, activation="relu"))
#nn_model.add(Dropout(0.2))
# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=512, activation="relu"))
#nn_model.add(Dropout(0.5))
# Third hidden layer
nn_model.add(tf.keras.layers.Dense(units=1024, activation="relu"))
#nn_model.add(Dropout(0.7))
# Third hidden layer
nn_model.add(tf.keras.layers.Dense(units=512, activation="sigmoid"))
#nn_model.add(Dropout(0.5))
nn_model.add(tf.keras.layers.Dense(units=64, activation="sigmoid"))
nn_model.add(Dropout(0.8))
# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

In [ ]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
fit_model = nn_model.fit(X_train, y_train, epochs=10)

In [ ]:
model_loss, model_accuracy = nn_model.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
predictions = nn_model.predict(X_test)
predictions

In [ ]:
X_test['Status'] = [0 if predection <0.5 else 1 for predection in predictions]

In [ ]:
y_test.describe()

In [ ]:
predictions_df = pd.DataFrame(predictions.tolist())
predictions_df.describe()

In [ ]:
predictions_df

In [ ]:
X_test

In [ ]:
nn_model.save('../nn_model.keras')

In [ ]:
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

from sklearn.linear_model import LogisticRegression

In [ ]:
model = LogisticRegression(random_state=1)

# Fit the model using training data
model.fit(X_train, y_train)

In [ ]:
model.score(X_train, y_train)

In [ ]:
# Make a prediction using the testing data
predictions = model.predict(X_test)
# Print the balanced_accuracy score of the model
balanced_accuracy_score(y_test,predictions)

In [ ]:
# Generate a confusion matrix for the model
confusion_matrix(y_test,predictions)

In [ ]:
# Print the classification report for the model
tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
accuracy = (tp + tn) / (tp + fp + tn + fn)
accuracy

In [ ]:
tp

In [ ]:
tn

In [ ]:
fp

In [ ]:
fn

In [ ]:
lr_predictions = model.predict(X_test)
lr_predictions

In [ ]:
X_test['Prediction'] = list(lr_predictions)

In [ ]:
X_test